In [1]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel

# LangChain OpenAI
from langchain_openai import ChatOpenAI
from whisper.utils import system_encoding

In [2]:
## .env 설정

load_dotenv()

True

In [3]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel

# LangChain OpenAI
from langchain_openai import ChatOpenAI

## 이미 생성된 합성 데이터셋 불러오기

In [4]:
## 이미 생성된 합성 데이터셋 생성
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_json("qa_dataset.jsonl", lines=True)
print(dataset_df.shape)
display(dataset_df)

(55, 3)


,question,reference,answer
0,인사 변화의 필요성을 분석하는 책임자는 누구인가요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 필요성 분석의 책임자는 인사팀입니다.
1,승진 및 이동 결정 시 어떤 기준을 수립해야 하나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,승진 및 이동 결정 시 후보자 선정 및 평가 기준을 CTO와 인사팀이 함께 수립해야...
2,인사 변화 공지는 언제 이루어지나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 공지는 2025-09-30에 인사팀에 의해 이루어집니다.
3,인사 변화 후 어떤 점을 모니터링해야 하나요?,[# 인사·조직 기밀 | 주요 인사(승진·이동·해고) 관리 문서\n\n작성일: 20...,인사 변화 후에는 조직 및 팀 성과에 미치는 영향을 지속적으로 모니터링해야 합니다.
4,조직 구조 개편안의 목적은 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,조직 구조 개편은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 위한 전략...
5,조직 구조 개편안의 적용 범위는 어떻게 되나요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,"적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)입니다."
6,조직 구조 개편안의 실행 계획에서 1단계는 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,"1단계는 현 조직 구조 분석 및 문제점 도출로, 기간은 2주이며 책임자는 인사팀입니다."
7,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 무엇인가요?,[# 인사·조직 기밀 | 조직 구조 개편안\n\n작성일: 2025-08-29\n회사...,직원 저항에 대한 대응 방안은 충분한 커뮤니케이션 및 교육 제공입니다.
8,핵심 인재 유출 리스크를 평가하기 위한 첫 번째 단계는 무엇인가요?,[# 인사·조직 기밀 | 핵심 인재 유출 리스크 분석 및 대응 전략\n\n작성일: ...,"첫 번째 단계는 리스크 진단이며, 목표는 현재 인재 유출 리스크를 평가하는 것입니다."
9,인재 유출을 예방하기 위한 조치에는 어떤 것들이 있나요?,[# 인사·조직 기밀 | 핵심 인재 유출 리스크 분석 및 대응 전략\n\n작성일: ...,"인재 유출을 예방하기 위한 조치로는 보상 체계 재검토 및 개선, 경력 개발 및 교육..."


## 우리가 쓰는 sllm RAG 체인 불러오기

In [5]:
from dotenv import load_dotenv

load_dotenv()

from pathlib import Path
import sys

root = Path.cwd()
while not (root / "utils_qwen3_8b").exists() and root != root.parent:
    root = root.parent
sys.path.insert(0, str(root))

from utils_qwen3_8b.rag import chat

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3


In [6]:

response = chat({"history": [
    {"role": "user", "content": "코드노바의 API 서버 기술스택알려줘"}], "tone":"formal", "permission":"cto"})

INFO:utils_qwen3_8b.service:-------- Chat Request - Permission: cto, Tone: formal
INFO:utils_qwen3_8b.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "cto_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
                </tools>

                For each function call, return a json object with function name and arg

In [7]:
print(response)

{'response': '코드노바의 API 서버 기술스택은 Node.js와 Express.js를 사용하고 있습니다.', 'result': ['검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->\n# 서비스 아키텍처 문서\n분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n## 1. 개요\n코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. API 서버\n- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.\n- **기술 스택**: Node.js, Express.js\n- **검증 포인트**:\n  - API 엔드포인트가 올바르게 작동하는지 확인\n  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]\n<!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->\n# 프론트엔드 아키텍처 문서\n분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n---\n\n## 1. 개요\n코드노바의 프론트엔드 아키텍처는 사용자 경험을 극대화하고, 유지보수성을 높이며, 확장성을 고려하여 설계되었습니다. 본 문서는 프론트엔드 개발자가 이해하고 실행할 수 있도록 아키텍처의 구성 요소와 원칙을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. 기술 스택\n- **프레임워크**: React, Vue.js 또는 Angular 중 하나를 선택하여 사용\n- **상태 관리**: Redux, Vuex 또는 Context API를 사용하여 애플리케이션 상태 관리\n- **스타일링

In [8]:
print(response['response'])

코드노바의 API 서버 기술스택은 Node.js와 Express.js를 사용하고 있습니다.


In [9]:
response = chat({"history": [
    {"role": "user", "content": "안녕하세요!"},
    {"role": "assistant", "content": "반가워요!"},
    {"role": "user", "content": "백엔드 팀 문서 알려줘"},
    {"role": "assistant", "content": "백엔드 팀 문서에는 주간 보고서가 있습니다."},
    {"role": "user", "content": "거기엔 무슨 내용이 있어?"},
    {"role": "assistant", "content": """백엔드팀 주간 업무 계획 문서에는 이렇게 적혀 있어.

1. 서비스 안정성 강화: 서버 모니터링 도구 점검, 주요 지표 확인, 이상 징후 대응 방안 마련
2. 신규 기능 개발 및 배포: 요구사항 정의서 작성, API 설계, 개발 일정 수립, 코드 리뷰 및 테스트, 배포 준비
3. 기술 부채 관리 및 리팩토링: 코드 리뷰 일정 수립, 기술 부채 리스트 업데이트

또, 주간 업무 계획에는 서비스 안정성 향상, 신규 기능 개발 로드맵, 기술 부채 관리 등 구체적인 업무 항목과 체크리스트가 포함돼 있어."""},
    {"role": "user", "content": "오늘 저녁 메뉴 추천해줄래?"},
    {"role": "assistant", "content": "오늘 저녁 메뉴로는 균형 잡힌 영양과 맛을 고려하여 닭가슴살 샐러드나 구운 채소와 함께하는 햄버거를 추천드립니다. 건강에도 좋고 간편하게 즐기기에도 적합합니다."},
    {"role": "user", "content": "안뇽"}
    ], "tone":"formal", "permission":"backend"})

INFO:utils_qwen3_8b.service:-------- Chat Request - Permission: backend, Tone: formal
INFO:utils_qwen3_8b.service:-------- Tools Prompt: 
                                사용자는 backend(백엔드)팀에 속한 팀원입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "backend_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
                </tools>

                For each function call, return a json object w

In [10]:
print(response['response'])

안녕하세요. 무엇을 도와드릴까요?


## 정성적 평가 체인 셋팅

In [11]:
def api_document_evaluation_chain_json():
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={"response_format": {"type": "json_object"}},
    )

    evaluation_prompt = PromptTemplate.from_template(
        """
    당신은 코드노바의 사내 문서 기반 평가 전문가입니다. 주어진 시스템의 답변이 **기준 정답과 얼마나 일치하는지**, **중요한 정보를 얼마나 잘 재현했는지**, 그리고 **세부 사항을 얼마나 잘 반영했는지** 평가해야 합니다.

    평가 기준:
    1. **정확도 (Accuracy)**: 시스템의 답변이 **기준 정답**과 얼마나 일치하는지 평가하세요.
        - 3점: **완벽하게 일치**. 구체적인 매개변수, 반환값, 예시까지 정확히 일치.
        - 2점: **일치**하나, 예시나 코드에서 **약간 차이**가 있을 때.
        - 1점: **핵심 내용**은 맞지만 **세부 내용**이 **누락되거나 잘못된 설명**.
        - 0점: **기준 정답과 크게 다름**. 예시나 코드에서 **큰 차이**가 있을 경우 점수는 **0점**.

    2. **재현율 (Recall)**: 시스템의 답변이 **기준 정답에서 제공한 중요한 정보**(예: 매개변수, 반환값 등)를 얼마나 잘 재현했는지 평가하세요.
        - 3점: **모든 핵심 정보**를 **정확하게 재현**.
        - 2점: **핵심 정보**는 잘 재현되었으나 **세부 내용**이 부족하거나 다르게 표현됨.
        - 1점: **핵심 정보**가 **누락**되었거나 **잘못된 설명**이 있음.
        - 0점: **핵심 정보**가 많이 **누락**됨.

    3. **구체성 (Specificity)**: 시스템의 답변이 **기준 정답에서 제공한 세부 사항**(매개변수 설명, 예시 코드 등)을 얼마나 구체적으로 반영했는지 평가하세요.
        - 3점: **매개변수**와 **반환값**을 **구체적으로 설명**하며, **예시 코드**나 **상세한 설명**을 포함.
        - 2점: **핵심 매개변수**는 설명되었지만, **구체적인 형식이나 예시 코드**가 부족.
        - 1점: **기본적인 설명**은 있지만 **구체적인 예시나 설명**이 부족하거나 불완전.
        - 0점: **구체적인 설명**이 부족하고, 중요한 **세부 사항**이 **누락**되었을 때 점수는 **0점**.

    ## 예시 1:
    **질문**: 코드노바의 `API 서버 기술 스택`은 무엇인가요?
    **기준 정답 (코드노바 사내 문서)**: "코드노바의 API 서버는 주로 Python, Django를 사용하며, 데이터베이스는 PostgreSQL을 사용합니다. 서버는 AWS EC2에서 호스팅되며, API 게이트웨이로 Nginx를 사용합니다. 클라우드 서비스는 AWS와 Google Cloud를 활용합니다."
    **시스템 답변**: "코드노바는 Django와 PostgreSQL을 사용하여 API 서버를 운영합니다. AWS와 Google Cloud를 이용하고, Nginx로 트래픽을 관리합니다."

    ### 평가:
    - **정확도 (Accuracy)**: 점수는 0에서 100 사이로 평가하세요. (예: 3)
    - **재현율 (Recall)**: 점수는 0에서 100 사이로 평가하세요. (예: 3)
    - **구체성 (Specificity)**: 점수는 0에서 100 사이로 평가하세요. (예: 2)

    ## 예시 2:
    **질문**: `getDistance` 메서드의 입력 매개변수는 무엇인가요?
    **기준 정답 (코드노바 사내 문서)**: "`getDistance` 메서드는 두 개의 매개변수를 받습니다: `origin` (경로의 시작점, 문자열)과 `destination` (경로의 종료점, 문자열)입니다. 이 메서드는 시작점과 종료점 간의 거리를 계산하여 반환합니다."
    **시스템 답변**: "`getDistance` 메서드는 `origin`과 `destination` 매개변수를 사용하여 거리를 계산합니다."

    ### 평가:
    - **정확도 (Accuracy)**: 점수는 0에서 100 사이로 평가하세요. (예: 3)
    - **재현율 (Recall)**: 점수는 0에서 100 사이로 평가하세요. (예: 3)
    - **구체성 (Specificity)**: 점수는 0에서 100 사이로 평가하세요. (예: 2)

    ## 사용자 질문 : {question}
    ## 기준 정답 : {answer}
    ## 시스템 답변 : {system_answer}

    시스템의 답변을 기반으로 정확도, 재현율, 구체성을 평가한 후, 각 항목에 대한 점수를 **0에서 100 사이**로 계산하여 다음 JSON 형태로 제공합니다:

    {{"accuracy": 3, "recall": 3, "specificity": 2}}
    """
    )

    evaluation_chain = evaluation_prompt | llm | StrOutputParser()
    return evaluation_chain


### 우리의 파인튜닝된 sllm의 RAG 답변을 배치로 정성적 평가

In [12]:
import json
import pandas as pd
import numpy as np

# JSON 응답을 받는 평가 체인 생성
evaluation_chain_json = api_document_evaluation_chain_json()

# 모든 데이터에 대해 평가 수행
evaluation_results = []

for idx in range(len(dataset_df)):
    question = dataset_df['question'][idx]
    answer = dataset_df['answer'][idx]
    
    # run_langraph로 시스템 답변 생성
    system_answer = chat({"history": [
    {"role": "user", "content": question}], "tone":"formal", "permission":"cto"})['response']
    
    # 평가 수행
    response = evaluation_chain_json.invoke({
        "question": question,
        "answer": answer,
        "system_answer": system_answer
    })
    
    # JSON 파싱
    try:
        scores = json.loads(response)
        evaluation_results.append({
            'index': idx,
            'question': question,
            'reference': answer,
            'system_answer': system_answer,
            'accuracy': scores.get('accuracy', 0),
            'recall': scores.get('recall', 0),
            'specificity': scores.get('specificity', 0)
        })
        print(f"평가 완료: {idx+1}/{len(dataset_df)} - Accuracy: {scores.get('accuracy', 0)}")
    except json.JSONDecodeError:
        print(f"JSON 파싱 오류: {idx+1}/{len(dataset_df)}")
        evaluation_results.append({
            'index': idx,
            'question': question,
            'reference': answer,
            'system_answer': system_answer,
            'accuracy': 0,
            'recall': 0,
            'specificity': 0
        })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(evaluation_results)

# 평균 점수 계산
avg_accuracy = results_df['accuracy'].mean()*100/3
avg_recall = results_df['recall'].mean()*100/3
avg_specificity = results_df['specificity'].mean()*100/3
overall_avg = (avg_accuracy + avg_recall + avg_specificity) / 3

print(f"\n=== 전체 평가 결과 ===")
print(f"평균 정확도: {avg_accuracy:.2f}")
print(f"평균 재현율: {avg_recall:.2f}")
print(f"평균 구체성: {avg_specificity:.2f}")
print(f"전체 평균: {overall_avg:.2f}")

# 결과 저장
results_df.to_csv('batch_evaluation_results.csv', index=False)
print(f"\n결과가 'batch_evaluation_results.csv'에 저장되었습니다.")


INFO:utils_qwen3_8b.service:-------- Chat Request - Permission: cto, Tone: formal
INFO:utils_qwen3_8b.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "cto_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
                </tools>

                For each function call, return a json object with function name and arg

평가 완료: 1/55 - Accuracy: 3


INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- LLM Tool Parse Response Success
INFO:utils_qwen3_8b.service:-------- LLM Tools Match : 2
INFO:utils_qwen3_8b.service:-------- cto search keyword: 승진 기준
INFO:utils_qwen3_8b.service:-------- cto search found 6 results.
INFO:utils_qwen3_8b.service:-------- cto_search tool Response Success
INFO:utils_qwen3_8b.service:-------- cto search keyword: 이동 결정 기준
INFO:utils_qwen3_8b.service:-------- cto search found 6 results.
INFO:utils_qwen3_8b.service:-------- cto_search tool Response Success
INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- Final Assistant Reply Generated
INFO:utils_qwen3_8b.service:-------- 승진 및 이동 결정 시 다음과 같은 기준을 수립하는 것이 권장됩니다.

1. 성과 기반 결정: 객관적인 성과 지표에 기반한 평가를 실시합니다.
2. 팀의 의견 수렴: 팀 리더 및 동료의 피드백을 충분히 반영합니다.
3. 인사 변화 후 모니터링: 결정

평가 완료: 2/55 - Accuracy: 1


INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- LLM Tool Parse Response Success
INFO:utils_qwen3_8b.service:-------- LLM Tools Match : 1
INFO:utils_qwen3_8b.service:-------- cto search keyword: 인사 변화 공지 일정
INFO:utils_qwen3_8b.service:-------- cto search found 6 results.
INFO:utils_qwen3_8b.service:-------- cto_search tool Response Success
INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- Final Assistant Reply Generated
INFO:utils_qwen3_8b.service:-------- 인사 변화 공지는 2025년 9월 30일에 이루어집니다.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- Chat Request - Permission: cto, Tone: formal
INFO:utils_qwen3_8b.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></too

...

평가 완료: 53/55 - Accuracy: 1


INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- LLM Tool Parse Response Success
INFO:utils_qwen3_8b.service:-------- LLM Tools Match : 2
INFO:utils_qwen3_8b.service:-------- cto search keyword: 핵심 인재 유지 전략 보고서 파일명
INFO:utils_qwen3_8b.service:-------- cto search found 6 results.
INFO:utils_qwen3_8b.service:-------- cto_search tool Response Success
INFO:utils_qwen3_8b.service:-------- cto search keyword: 핵심 인재 스카우트 전략 보고서 파일명
INFO:utils_qwen3_8b.service:-------- cto search found 6 results.
INFO:utils_qwen3_8b.service:-------- cto_search tool Response Success
INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- Final Assistant Reply Generated
INFO:utils_qwen3_8b.service:-------- 핵심 인재 유지 및 스카우트 전략 보고서는 사내 문서로 "02_인사_조직_기밀__핵심_인재_유지_및_스카우트_전략_보고서.txt" 파일명으로 저장되어 있습니다.
INFO:httpx:HTTP Reques

평가 완료: 54/55 - Accuracy: 3


INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- LLM Tool Parse Response Success
INFO:utils_qwen3_8b.service:-------- LLM Tools Match : 1
INFO:utils_qwen3_8b.service:-------- cto search keyword: 조직 구조 개편안 문서 저장 위치
INFO:utils_qwen3_8b.service:-------- cto search found 6 results.
INFO:utils_qwen3_8b.service:-------- cto_search tool Response Success
INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils_qwen3_8b.service:-------- Final Assistant Reply Generated
INFO:utils_qwen3_8b.service:-------- 조직 구조 개편안 문서는 CodeNova 사내 문서 중 "인사·조직 기밀" 분류에 속하며, 구체적으로 "04_인사_조직_기밀__조직_구조_개편안.txt" 파일로 저장되어 있습니다.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


평가 완료: 55/55 - Accuracy: 3

=== 전체 평가 결과 ===
평균 정확도: 70.91
평균 재현율: 74.55
평균 구체성: 65.45
전체 평균: 70.30

결과가 'batch_evaluation_results.csv'에 저장되었습니다.


In [13]:
result = pd.read_csv('batch_evaluation_results.csv')

display(result)

,index,question,reference,system_answer,accuracy,recall,specificity
0,0,인사 변화의 필요성을 분석하는 책임자는 누구인가요?,인사 변화 필요성 분석의 책임자는 인사팀입니다.,인사 변화의 필요성을 분석하는 책임자는 인사팀입니다.,3,3,3
1,1,승진 및 이동 결정 시 어떤 기준을 수립해야 하나요?,승진 및 이동 결정 시 후보자 선정 및 평가 기준을 CTO와 인사팀이 함께 수립해야...,승진 및 이동 결정 시 다음과 같은 기준을 수립하는 것이 권장됩니다.\n\n1. 성...,1,1,1
2,2,인사 변화 공지는 언제 이루어지나요?,인사 변화 공지는 2025-09-30에 인사팀에 의해 이루어집니다.,인사 변화 공지는 2025년 9월 30일에 이루어집니다.,3,3,3
3,3,인사 변화 후 어떤 점을 모니터링해야 하나요?,인사 변화 후에는 조직 및 팀 성과에 미치는 영향을 지속적으로 모니터링해야 합니다.,인사 변화 후에는 다음과 같은 점을 모니터링해야 합니다.\n\n1. 인사 변화가 조...,3,3,2
4,4,조직 구조 개편안의 목적은 무엇인가요?,조직 구조 개편은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 위한 전략...,조직 구조 개편안의 목적은 CodeNova의 성장과 변화하는 시장 환경에 적응하기 ...,3,3,3
5,5,조직 구조 개편안의 적용 범위는 어떻게 되나요?,"적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)입니다.","조직 구조 개편안의 적용 범위는 모든 부서(개발, 마케팅, 운영, 인사)이며, 대상...",3,3,2
6,6,조직 구조 개편안의 실행 계획에서 1단계는 무엇인가요?,"1단계는 현 조직 구조 분석 및 문제점 도출로, 기간은 2주이며 책임자는 인사팀입니다.","조직 구조 개편안의 실행 계획에서 1단계는 ""현 조직 구조 분석 및 문제점 도출""입...",3,3,2
7,7,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 무엇인가요?,직원 저항에 대한 대응 방안은 충분한 커뮤니케이션 및 교육 제공입니다.,조직 구조 개편안에서 직원 저항에 대한 대응 방안은 다음과 같습니다.\n\n- 충분...,3,3,3
8,8,핵심 인재 유출 리스크를 평가하기 위한 첫 번째 단계는 무엇인가요?,"첫 번째 단계는 리스크 진단이며, 목표는 현재 인재 유출 리스크를 평가하는 것입니다.","핵심 인재 유출 리스크 평가의 첫 번째 단계는 ""리스크 진단""입니다. 이 단계에서는...",3,3,2
9,9,인재 유출을 예방하기 위한 조치에는 어떤 것들이 있나요?,"인재 유출을 예방하기 위한 조치로는 보상 체계 재검토 및 개선, 경력 개발 및 교육...",인재 유출을 예방하기 위한 조치는 다음과 같습니다.\n\n1. 보상 체계 재검토 및...,2,2,2
